In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

import pytorch_grad_cam
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
import pickle
import numpy as np

In [19]:
batch_size = 16

transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

Files already downloaded and verified
Files already downloaded and verified


In [20]:
model = torchvision.models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 10)  
checkpoint = torch.load('./checkpoints/resnet18_cifar10_epho10.pth')
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [21]:
save_data = []

### 保存测试集中的img,label，保存训练集只需要把test_loader换成train_loader

In [22]:
save_ori_data = []
for batch_idx, (images, labels) in enumerate(test_loader):
    # images, labels = images.to(device), labels.to(device)
    images_np = images.cpu().numpy()
    labels_np = labels.cpu().numpy()
    batch_data = (images_np, labels_np)
    save_ori_data.extend([batch_data])

# 将每个批次的数据拼接成一个大的数组
concatenated_images = np.concatenate([item[0] for item in save_ori_data], axis=0)

# 将每个批次的 labels 拼接成一个大的数组
concatenated_labels = np.concatenate([item[1] for item in save_ori_data], axis=0)

# 重新构建 save_data 列表
save_ori_data = [(concatenated_images, concatenated_labels)]

with open('./data/test_resize.pkl', 'wb') as file:
    pickle.dump(save_ori_data, file)


### 保存测试集中的ground_truth，保存训练集只需要把test_loader换成train_loader

In [23]:
target_layers = [model.layer4[-1]]
cam = GradCAM(model=model, target_layers=target_layers, use_cuda=True)

save_gound_truth_data = []

for batch_idx, (images, labels) in enumerate(test_loader):
    images_np = images.cpu().numpy()
    labels_np = labels.cpu().numpy()
    targets = [ClassifierOutputTarget(label) for label in labels]
    ground_truth = cam(input_tensor=images,targets=targets)

    save_gound_truth_data.extend([ground_truth])

concatenated_ground_truth = np.concatenate(save_gound_truth_data, axis=0)
save_data = [(concatenated_ground_truth, save_gound_truth_data[0][1])]


with open('./data/test_ground_truth_resize.pkl', 'wb') as file:

    pickle.dump(save_gound_truth_data, file) 

### 将测试集/训练集拼接成（img，label，ground_truth）

In [25]:
import pickle
import numpy as np

# 加载 test_resize.pkl 的数据
with open('./data/test_resize.pkl', 'rb') as file:
    loaded_data_resize = pickle.load(file)

# 加载 test_ground_truth_resize.pkl 的数据
with open('./data/test_ground_truth_resize.pkl', 'rb') as file:
    loaded_data_ground_truth = pickle.load(file)

# 将 test_ground_truth_resize.pkl 中的 loaded_data[0][0] 放入 test_resize.pkl 中的 loaded_data[0][2]
loaded_data_resize[0] = (loaded_data_resize[0][0], loaded_data_resize[0][1], loaded_data_ground_truth[0][0])

# 保存修改后的数据
with open('./data/test_resize_modified.pkl', 'wb') as file:
    pickle.dump(loaded_data_resize, file)
